In [16]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [17]:
# settings
np.random.seed(2020)
name = "Dblp"
dim = 128
come_model_type = "GMM"
ks = [2, 5, 10, 20]

In [18]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [19]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_ds-0.0_d-{dim}_type-{come_model_type}_k-{k}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_d{dim}_k{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/minico

k = 2
k = 5
k = 10
k = 20


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.364125,-0.588987,-0.409808,0.565156,0.283277,-0.055052,0.010457,0.394754,0.167438,0.387936,...,0.002374,-0.049337,-0.318054,-0.146058,-0.548859,0.118959,-0.110706,-0.174423,0,0
1,0.278587,-0.588879,-0.333209,0.524908,0.227196,-0.124308,0.014563,0.391919,-0.003007,0.488111,...,-0.049671,-0.115132,-0.349642,-0.048473,-0.607197,0.024172,0.005262,-0.032287,0,2
2,0.465188,-0.502005,-0.437610,0.589219,0.391638,0.025219,-0.062833,0.402654,0.273840,0.392184,...,-0.033287,0.038274,-0.330315,-0.181933,-0.530037,0.043615,-0.040437,-0.174863,0,2
3,0.337499,-0.522336,-0.434716,0.504707,0.202075,-0.109289,0.060233,0.377867,0.216318,0.324594,...,-0.072317,-0.078721,-0.274162,-0.221168,-0.387503,0.047380,-0.008381,-0.216016,0,0
4,0.231919,-0.461826,-0.440046,0.581936,0.269956,-0.062691,-0.087539,0.398939,0.309795,0.334096,...,0.010352,-0.057188,-0.280246,-0.229399,-0.395916,0.004763,-0.106810,-0.201934,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.256909,-0.432436,-0.269539,0.371987,0.343081,-0.054403,-0.018651,0.347056,0.179956,0.270165,...,-0.172171,-0.030358,-0.255791,-0.221646,-0.246279,0.019650,0.086308,0.023113,0,0
13180,0.046119,-0.524076,-0.319708,-0.063703,0.488777,0.066487,0.056589,0.085111,0.315828,0.430509,...,-0.287074,0.299555,-0.042789,0.201546,-0.090146,-0.426288,0.044974,0.343586,0,3
13181,-0.015298,-0.416248,-0.166379,-0.049717,0.388531,-0.009873,-0.013568,-0.017180,0.400259,0.496808,...,-0.089020,0.311606,-0.170968,0.110497,-0.233802,-0.443860,0.005674,0.337000,0,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.030520,-0.303298,-0.052067,0.216470,0.303616,-0.087638,0.102583,0.246364,0.196837,0.426157,...,-0.143494,0.195385,-0.206445,-0.020120,-0.057692,-0.027288,0.124423,0.182465,0.300061,2.190382
std,0.178880,0.189579,0.179750,0.164567,0.145128,0.214682,0.159730,0.176149,0.194746,0.164508,...,0.146441,0.146658,0.168963,0.147867,0.188953,0.159465,0.222875,0.187429,0.458301,1.315339
min,-0.616642,-0.957211,-0.609815,-0.401549,-0.193860,-0.683715,-0.434596,-0.491078,-0.386489,-0.243133,...,-0.683022,-0.361166,-0.803104,-0.628961,-0.759284,-0.640512,-0.633719,-0.367539,0.000000,0.000000
25%,-0.085560,-0.445479,-0.175488,0.106477,0.203967,-0.243855,-0.003986,0.128458,0.054806,0.324486,...,-0.238445,0.095248,-0.324347,-0.115016,-0.192072,-0.125442,-0.035548,0.037936,0.000000,1.000000
50%,0.036057,-0.298440,-0.064065,0.223441,0.303956,-0.107959,0.095558,0.239322,0.183981,0.431529,...,-0.134402,0.195295,-0.208296,-0.018903,-0.073063,-0.022658,0.089545,0.177880,0.000000,3.000000
75%,0.152158,-0.165985,0.052499,0.332915,0.398696,0.059170,0.204077,0.360693,0.326738,0.537919,...,-0.041417,0.290757,-0.083870,0.077095,0.061411,0.076171,0.260792,0.317644,1.000000,3.000000
max,0.650580,0.462497,0.661576,0.811872,0.920628,0.602605,0.823321,0.856029,0.885655,0.939497,...,0.392417,0.771999,0.330308,0.556829,0.572088,0.590620,0.960098,0.779552,1.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.320240,-0.627375,-0.293566,0.448015,0.329800,-0.009727,-0.110384,0.402813,0.155657,0.299463,...,0.107143,-0.117253,-0.277003,-0.033361,-0.499929,0.156617,-0.091086,-0.328594,0,0
1,0.211466,-0.592742,-0.253647,0.456053,0.263981,-0.157476,-0.058294,0.403993,0.033025,0.501744,...,0.129525,-0.202995,-0.290447,0.087044,-0.600270,0.023531,0.107657,-0.149413,0,2
2,0.462118,-0.425457,-0.407647,0.470694,0.416651,0.035922,-0.141681,0.403183,0.330284,0.223219,...,0.011425,0.116289,-0.301078,-0.153339,-0.477320,0.054065,-0.057761,-0.287782,0,2
3,0.337203,-0.495905,-0.361851,0.393133,0.194511,-0.047649,-0.033379,0.325715,0.244276,0.163284,...,0.034075,-0.033329,-0.209047,-0.217807,-0.384692,-0.003024,0.035391,-0.335863,0,0
4,0.170511,-0.513423,-0.320501,0.437178,0.184796,-0.006720,-0.220844,0.281199,0.331932,0.205735,...,0.176548,-0.020387,-0.208459,-0.156927,-0.341746,-0.059401,-0.087415,-0.336903,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.246386,-0.365368,-0.222700,0.235965,0.301695,-0.024480,-0.144608,0.428081,0.277141,0.204780,...,-0.111092,-0.138256,-0.125790,-0.197135,-0.193298,-0.069245,0.161825,-0.109317,0,0
13180,-0.064219,-0.483084,-0.233587,-0.088730,0.516231,0.078248,0.099956,-0.016751,0.389546,0.485400,...,-0.355484,0.319764,0.069125,0.248837,-0.009021,-0.526786,0.145140,0.459908,3,3
13181,-0.137615,-0.401715,-0.108275,-0.083856,0.492230,0.009986,0.073602,-0.072570,0.482359,0.525593,...,-0.162884,0.353830,-0.047686,0.146902,-0.133835,-0.515325,0.135428,0.483413,3,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.023101,-0.265136,-0.035295,0.212223,0.254659,-0.098608,0.066606,0.241891,0.151294,0.392008,...,-0.114951,0.162886,-0.184760,-0.001191,-0.033928,-0.071781,0.082733,0.173346,1.972239,2.190382
std,0.182738,0.192924,0.186552,0.178109,0.169988,0.219832,0.184243,0.201588,0.212019,0.172438,...,0.162310,0.168625,0.199254,0.169353,0.172430,0.193864,0.199098,0.228917,1.314581,1.315339
min,-0.682202,-0.899385,-0.661957,-0.434804,-0.463454,-0.706818,-0.532346,-0.505480,-0.457103,-0.371703,...,-0.747638,-0.466403,-0.837697,-0.691032,-0.726374,-0.746128,-0.611348,-0.455965,0.000000,0.000000
25%,-0.087918,-0.410200,-0.161362,0.087594,0.139649,-0.257137,-0.059312,0.104573,-0.005469,0.281268,...,-0.217904,0.049465,-0.329664,-0.109807,-0.154040,-0.190664,-0.056854,-0.002695,1.000000,1.000000
50%,0.024684,-0.267749,-0.042847,0.217226,0.253085,-0.110718,0.062604,0.231490,0.138165,0.394746,...,-0.107872,0.162359,-0.175078,0.000159,-0.043650,-0.070759,0.069898,0.163397,2.000000,3.000000
75%,0.142546,-0.124269,0.077922,0.333309,0.366787,0.051912,0.187477,0.374821,0.292931,0.507272,...,-0.000314,0.274700,-0.035551,0.107208,0.081088,0.058603,0.203143,0.340189,3.000000,3.000000
max,0.728275,0.453631,0.750070,0.908127,0.953793,0.727614,0.798593,0.940968,1.016743,0.952634,...,0.525707,0.816107,0.405745,0.706865,0.597016,0.638787,0.835116,0.845372,4.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.315910,-0.532388,-0.379827,0.448640,0.222977,-0.046450,-0.040365,0.420613,0.228489,0.267336,...,0.059782,-0.055670,-0.302633,-0.086992,-0.572689,0.084252,-0.007192,-0.289730,2,0
1,0.154762,-0.531019,-0.225194,0.448152,0.191505,-0.026764,0.044427,0.470409,0.064429,0.495237,...,0.094173,-0.283913,-0.219839,0.007076,-0.647662,-0.045071,0.141692,-0.109914,2,2
2,0.479796,-0.397699,-0.386404,0.516044,0.331575,0.018234,-0.172640,0.337710,0.335203,0.350686,...,-0.034158,0.039443,-0.286122,-0.177710,-0.483438,0.058460,0.099476,-0.321283,2,2
3,0.303357,-0.466494,-0.420090,0.380732,0.131113,-0.071237,0.028744,0.319232,0.271011,0.135511,...,-0.005327,-0.054087,-0.280375,-0.260676,-0.368144,0.004691,0.051465,-0.367432,2,0
4,0.074202,-0.403759,-0.405593,0.426719,0.117810,0.034774,-0.089488,0.321120,0.308847,0.083192,...,-0.005842,-0.119104,-0.278471,-0.270200,-0.341454,-0.050394,-0.055726,-0.351128,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.272360,-0.296185,-0.248778,0.336725,0.250997,0.060097,-0.188221,0.365663,0.190870,0.243973,...,-0.171256,-0.162119,-0.258160,-0.263908,-0.167533,-0.007468,0.331365,-0.182841,2,0
13180,-0.016705,-0.391515,-0.219307,-0.124734,0.362215,0.143150,0.100554,0.022434,0.336065,0.410626,...,-0.372643,0.403307,0.027753,0.491067,0.052873,-0.738304,0.131836,0.252211,6,3
13181,0.046126,-0.293869,-0.192773,-0.175473,0.513854,0.152859,-0.023165,-0.142364,0.383600,0.542746,...,-0.177827,0.328586,-0.089681,0.242109,-0.127021,-0.685420,0.135028,0.135435,6,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.029747,-0.253318,-0.039365,0.216951,0.255441,-0.084419,0.098946,0.224237,0.154333,0.390469,...,-0.122270,0.156087,-0.180140,-0.010186,-0.033723,-0.059663,0.089582,0.154011,4.302715,2.190382
std,0.187380,0.195677,0.194601,0.183860,0.181825,0.205056,0.196257,0.201623,0.209521,0.180574,...,0.167694,0.182738,0.193114,0.184613,0.178618,0.198299,0.212159,0.211792,2.944180,1.315339
min,-0.615881,-0.916256,-0.675692,-0.388326,-0.390051,-0.705681,-0.560842,-0.493095,-0.459716,-0.309091,...,-0.806013,-0.512130,-0.836094,-0.766832,-0.851865,-0.848167,-0.630505,-0.536418,0.000000,0.000000
25%,-0.087221,-0.394685,-0.170342,0.093080,0.130855,-0.236933,-0.037400,0.082659,0.004464,0.273341,...,-0.227500,0.034995,-0.310928,-0.127332,-0.158979,-0.185854,-0.058167,-0.003044,2.000000,1.000000
50%,0.030819,-0.250671,-0.045833,0.222141,0.252518,-0.089321,0.097494,0.215378,0.141799,0.389881,...,-0.112584,0.158235,-0.168999,-0.003780,-0.041919,-0.057190,0.068558,0.150774,4.000000,3.000000
75%,0.154839,-0.114103,0.085472,0.344007,0.374184,0.059435,0.230851,0.358927,0.291165,0.507975,...,-0.004285,0.276979,-0.041281,0.113303,0.083988,0.070884,0.215672,0.308439,7.000000,3.000000
max,0.704828,0.419138,0.838114,0.895329,1.065553,0.676855,0.767923,1.003888,1.005607,0.997738,...,0.423483,0.834004,0.407396,0.707926,0.624985,0.641626,0.874921,0.832917,9.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.436527,-0.564456,-0.454166,0.641066,0.168700,0.082310,-0.031954,0.361073,-0.208672,0.276056,...,0.050538,-0.224988,-0.431624,0.003662,-0.477081,0.121960,0.052909,-0.350542,1,0
1,0.403277,-0.511639,-0.333275,0.545582,0.051169,0.028561,-0.091227,0.370079,-0.289292,0.483094,...,-0.015528,-0.363553,-0.475505,0.014565,-0.530358,-0.040160,0.263523,-0.140697,1,2
2,0.487985,-0.513284,-0.554191,0.689418,0.271111,0.052279,-0.025626,0.387778,-0.092199,0.317105,...,-0.009474,-0.074029,-0.445090,-0.088949,-0.461879,0.054054,0.101626,-0.306905,1,2
3,0.424656,-0.452077,-0.523225,0.574095,0.080523,-0.010256,0.022688,0.317095,-0.171146,0.266117,...,-0.084191,-0.234062,-0.329075,-0.094375,-0.241655,0.048418,0.168131,-0.347555,1,0
4,0.372900,-0.446677,-0.523795,0.631834,0.126949,0.054397,-0.117205,0.325753,-0.081025,0.227695,...,0.068887,-0.225661,-0.478980,-0.154673,-0.311928,-0.065331,0.082394,-0.291411,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.374316,-0.279774,-0.335749,0.397761,0.261890,-0.008758,-0.188682,0.413393,-0.071640,0.283936,...,-0.216151,-0.344590,-0.358659,-0.285082,-0.202561,-0.054232,0.287505,-0.183615,1,0
13180,0.054672,-0.411832,-0.357445,-0.168166,0.181663,0.170066,-0.083065,0.187582,0.377385,0.344669,...,-0.356740,0.299442,-0.133717,0.038592,0.229791,-0.662603,0.141447,0.282151,8,3
13181,-0.020217,-0.474941,-0.114838,0.143250,0.312997,-0.027986,-0.050794,-0.158910,0.536100,0.444589,...,0.131581,0.298015,-0.208109,0.016640,-0.277913,-0.598272,0.087692,0.395714,10,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.000830,-0.270871,-0.011392,0.247383,0.258536,-0.116252,0.136543,0.129188,0.110883,0.370419,...,-0.133110,0.136703,-0.151722,-0.037213,-0.070353,-0.016308,0.086585,0.091487,8.838365,2.190382
std,0.189273,0.221827,0.219088,0.210020,0.186759,0.217118,0.205720,0.212195,0.215034,0.175124,...,0.177751,0.208749,0.211260,0.179676,0.208271,0.186768,0.217748,0.217452,5.582605,1.315339
min,-0.760121,-1.043094,-0.790480,-0.482115,-0.385856,-0.820103,-0.522908,-0.544886,-0.563251,-0.230095,...,-0.751202,-0.681993,-0.866390,-0.900184,-0.804824,-0.864746,-0.581872,-0.646253,0.000000,0.000000
25%,-0.125075,-0.430068,-0.155517,0.103405,0.134624,-0.276165,-0.008557,-0.012369,-0.048370,0.253831,...,-0.248624,-0.003494,-0.301003,-0.141389,-0.221836,-0.136753,-0.066092,-0.063615,4.000000,1.000000
50%,-0.005971,-0.275395,-0.017926,0.252129,0.252892,-0.121410,0.134579,0.120678,0.090903,0.374856,...,-0.129734,0.141454,-0.148097,-0.028531,-0.077440,-0.014440,0.067656,0.097432,9.000000,3.000000
75%,0.132399,-0.114734,0.127231,0.389884,0.372108,0.028221,0.280896,0.264553,0.252303,0.487054,...,-0.011120,0.280569,-0.007655,0.077459,0.069192,0.108009,0.215292,0.249464,14.000000,3.000000
max,0.701677,0.452445,0.678930,1.098140,0.925440,0.663360,0.771111,0.954931,0.978782,1.002611,...,0.470287,0.805955,0.601112,0.714436,0.653253,0.684004,0.912726,0.733946,19.000000,4.000000


# Community Detection

In [20]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        if len(nodes) == 0: continue
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.02072897576868255
  i1 conductance: 0.02072897576868255
=>conductance: 0.02072897576868255
K = 5
  i0 conductance: 0.048034069525601665
  i1 conductance: 0.2009300162752848
  i2 conductance: 0.013701852396354014
  i3 conductance: 0.09629703166881763
  i4 conductance: 0.10914950259212554
=>conductance: 0.013701852396354014
K = 10
  i0 conductance: 0.29387417218543044
  i1 conductance: 0.26765849081892507
  i2 conductance: 0.15665179742849647
  i3 conductance: 0.3562529274004684
  i4 conductance: 0.23709513048872347
  i5 conductance: 0.47009045812664135
  i6 conductance: 0.49870801033591733
  i7 conductance: 0.08637822962849717
  i8 conductance: 0.38715369684136225
  i9 conductance: 0.06247833021288399
=>conductance: 0.06247833021288399
K = 20
  i0 conductance: 0.4314936318023929
  i1 conductance: 0.28
  i2 conductance: 0.4953125
  i3 conductance: 0.4541284403669725
  i4 conductance: 0.3968934118907338
  i5 conductance: 0.5799713876967096
  i6 conductance: 0.217

In [21]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.4802479401582417
K = 5
NMI:  0.6746848015894669
K = 10
NMI:  0.4963225850192131
K = 20
NMI:  0.3018911505699723


# Node Classification

In [22]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0, dim) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.1f}")
    print(f"  macro_f1: {macro_f1*100:.1f}")

k = 2
  micro_f1: 92.7
  macro_f1: 92.7
k = 5
  micro_f1: 92.6
  macro_f1: 92.4
k = 10
  micro_f1: 93.3
  macro_f1: 93.0
k = 20
  micro_f1: 92.4
  macro_f1: 92.2
